# Description

This is the inference of the [trained LSTM&CNN](https://www.kaggle.com/code/kostiantynlavronenko/dfl-bundesliga-simple-cnn-lstm) model. Here I predict the events for every second of the given test data. Since it was not specified exactly how big is the train dataset I set the counter MAX_TIME_TO_RUN to prevent that the runtime of the notebook exceeds the limits of the competition (maximum is 9hrs, but I set to 6 just for the example).

# Import libraries

In [1]:
import os
import subprocess
import gc
import cv2
import copy
import time
import random
import string
import joblib
import numpy as np 
import pandas as pd 
import torch
import time
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from torch.optim import lr_scheduler

import warnings
warnings.filterwarnings("ignore")


pd.options.display.max_colwidth = 1000

gc.enable()
target_size = "384x216"  # 20% of FullHD
os.environ["NVIDIA_VISIBLE_DEVICES"] = "all"
TIME_START = time.time()
MAX_TIME_TO_RUN = 3600*6

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Looking at the durations of all given test videos

In [3]:
test_paths = glob.glob("/kaggle/input/dfl-bundesliga-data-shootout/test/*.mp4")
N_files = len(test_paths)

In [4]:
durations = []
for path in test_paths:
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    durations.append(frame_count/fps)

# Creating test_df

In [5]:
times = []
paths = []
video_ids = []
for time_to_end, path in zip(durations,test_paths):
    TIME_SPLIT = 1 # 25 frames
    time_line = np.arange(0,time_to_end,TIME_SPLIT)
    video_id = os.path.splitext(os.path.basename(path))[0]
    times.extend(time_line)
    paths.extend([path]*len(time_line))
    video_ids.extend([video_id]*len(time_line))
D = {"video_id": video_ids,"time":times, "file_path":paths}
df_test = pd.DataFrame(D)
df_test.iloc[25:35]

,video_id,time,file_path
25,9d3c239b_0,25.0,/kaggle/input/dfl-bundesliga-data-shootout/test/9d3c239b_0.mp4
26,9d3c239b_0,26.0,/kaggle/input/dfl-bundesliga-data-shootout/test/9d3c239b_0.mp4
27,9d3c239b_0,27.0,/kaggle/input/dfl-bundesliga-data-shootout/test/9d3c239b_0.mp4
28,9d3c239b_0,28.0,/kaggle/input/dfl-bundesliga-data-shootout/test/9d3c239b_0.mp4
29,9d3c239b_0,29.0,/kaggle/input/dfl-bundesliga-data-shootout/test/9d3c239b_0.mp4
30,b2939d3c_0,0.0,/kaggle/input/dfl-bundesliga-data-shootout/test/b2939d3c_0.mp4
31,b2939d3c_0,1.0,/kaggle/input/dfl-bundesliga-data-shootout/test/b2939d3c_0.mp4
32,b2939d3c_0,2.0,/kaggle/input/dfl-bundesliga-data-shootout/test/b2939d3c_0.mp4
33,b2939d3c_0,3.0,/kaggle/input/dfl-bundesliga-data-shootout/test/b2939d3c_0.mp4
34,b2939d3c_0,4.0,/kaggle/input/dfl-bundesliga-data-shootout/test/b2939d3c_0.mp4


# Dataset

In [6]:
class VideoFramePredictDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.frames_split = 25 * TIME_SPLIT # 25 frames per second
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        file_path = self.df.iloc[index].file_path
        t = self.df.iloc[index].time
        video_id = self.df.iloc[index].video_id
        video_clip = np.zeros((self.frames_split, 128, 128, 3))
        try:
            cap = cv2.VideoCapture(file_path)
        except:
            return np.zeros((self.frames_split, 3, 128, 128))
        try:
            cap.set(cv2.CAP_PROP_POS_FRAMES,t*self.frames_split)
        except:
            pass
        for i in range(self.frames_split):
            try:
                _, frame = cap.read()
            except:
                frame = np.zeros((128,128,3))
            try:
                frame = cv2.resize(frame, (128,128))
            except:
                frame = np.zeros((128,128,3))
            video_clip[i] = frame
        try:
            video_clip = video_clip.transpose(0, 3, 1, 2)
        except:
            video_clip = np.zeros((self.frames_split, 3, 128, 128))
        return torch.from_numpy(video_clip), t, video_id

# Loading the model

## Architecture

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv3 = nn.Conv2d(20, 30, 5)
        
    def forward(self, i):
        x = i.view(-1, i.shape[2], i.shape[3], i.shape[4])
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = nn.AvgPool2d(4)(x)
        x = x.view(i.shape[0], i.shape[1], -1)
        return x
    
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(25230, 100)
        self.fc = nn.Linear(2500, 5)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x    

class LSTM_and_CNN(nn.Module):
    def __init__(self):
        super(LSTM_and_CNN, self).__init__()
        self.model_cnn = CNN().to(device)
        self.model_lstm = LSTM().to(device)
    
    def forward(self, x):
        features = self.model_cnn(x)
        out = self.model_lstm(features)
        return out

## Load the model and Dataset

In [8]:
#model = torch.jit.load('../input/dfl-bundesliga-simple-cnn-lstm/lstm_cnn.pth').to(device)
model = LSTM_and_CNN().to(device)
model.load_state_dict(torch.load('../input/dfl-bundesliga-simple-cnn-lstm/lstm_cnn.pth', map_location=device))
model.eval()

batch_size = 1
test_loader = DataLoader(
    VideoFramePredictDataset(df_test), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=1
)

# Predicting

In [9]:
def predict(model, dataloader):
    model.to(device)
    model.eval()
    dataloader = dataloader
    outputs = []
    ids = []
    times = []
    s = nn.Softmax(dim=1).to(device)
    for video_clip, t, video_id in tqdm(dataloader, leave=False):
        if time.time() - TIME_START > MAX_TIME_TO_RUN:
            break
        times.append(int(t))
        ids.append(video_id[0])
        try:
            frames = video_clip.to(device).float()
            output = model(frames)
            outputs.append(s(output.cpu()[:,:5])[0].detach().numpy())
        except Exception as e:
            outputs.append(s(torch.tensor([[1, 1, 1 ,1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), times, ids

In [ ]:
scores, times, ids = predict(model, test_loader)

  0%|          | 0/960 [00:00<?, ?it/s]

# Submitting 🚀

In [ ]:
submission_example = pd.read_csv("../input/dfl-bundesliga-data-shootout/sample_submission.csv")
submission_example.head()

I throw out the events **end** and **start** and than assign to each second one of the relevant events (so **challenge, throwin, play**). The reason why I can do this is because at th scoring only those times will be considered, that are near to the solution, the other will be rejected. At least it is what is said in the description of the Competition evaluation😝

In [ ]:
def func(x):
    D = {0:"challenge", 1:"throwin", 2:"play"}
    max_arg = np.argmax(x)
    return D[max_arg], x[max_arg]
D = {0:"challenge", 1:"throwin", 2:"play"}
scores_new = np.delete(scores, [3,4], axis = 1)
out_list = list(map(func, scores_new))
pred_events = []
pred_scores = []
for row in out_list:
    pred_events.append(row[0])
    pred_scores.append(row[1])

In [ ]:
D_dataframe = {"video_id": ids, "time":times, "event":pred_events, "score":pred_scores}
submission = pd.DataFrame(D_dataframe)
submission.head()

In [ ]:
submission.to_csv("submission.csv", index = False)